In [ ]:
import shutil
!pip install pygit2==1.12.2
%cd /content
folder_path = "/content/PlaylistCreator"
shutil.rmtree(folder_path, ignore_errors=True)
!git clone https://github.com/thefffilo/PlaylistCreator.git
%cd /content/PlaylistCreator/frontend
!npm i
!npm run build

In [ ]:
from google.colab import output
output.serve_kernel_port_as_window(8084)

## LLM

### Imports

In [ ]:
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

In [ ]:
!pip uninstall torch -y
!pip install torch==2.1

In [ ]:
!pip install pytorch-transformers
# !pip install transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
/usr/local/lib/python3.10/dist-packages/_XLAC.cpython-310-x86_64-linux-gnu.so: undefined symbol: _ZN3c109TupleTypeC1ESt6vectorINS_4Type24SingletonOrSharedTypePtrIS2_EESaIS4_EESt8optionalINS_13QualifiedNameEESt10shared_ptrINS_14FunctionSchemaEE

### Load model

In [ ]:
# load model from hub
from peft import PeftModel, PeftConfig

model_name = "fffilo/mistral-genre-classificator"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

config = PeftConfig.from_pretrained("fffilo/mistral-genre-classificator")
model = PeftModel.from_pretrained(model, "fffilo/mistral-genre-classificator")

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


In [ ]:
#Codice per estrarre solo i primi 3 generi utili
#Utile come parser dell'output del LLM, in questo modo viene standardizzato
#(nell'array 'genres' non ci sono tutti i generi ma solo alcuni di esempio)

import re

def extract_genres(stringa):
    genres = ["acoustic","afrobeat","alt-rock","alternative","ambient","anime","black-metal","blues","bossanova","brazil","breakbeat","british","children","chill","classical","club","comedy","country","dance","dancehall","death-metal","deep-house","disco","disney","drum-and-bass","dub","dubstep","edm","electro","electronic","emo","folk","french","funk","garage","german","gospel","goth","groove","grunge","guitar","happy","hard-rock","hardcore","hardstyle","heavy-metal","hip-hop","holidays","house","idm","indian","indie","indie-pop","industrial","j-pop","jazz","k-pop","kids","latin","latino","metal","metal-misc","metalcore","minimal-techno","movies","new-age","opera","party","piano","pop","pop-film","post-dubstep","power-pop","progressive-house","psych-rock","punk","punk-rock","r-n-b","rainy-day","reggae","reggaeton","road-trip","rock","rock-n-roll","rockabilly","romance","sad","salsa","samba","show-tunes","singer-songwriter","sleep","songwriter","soul","soundtracks","spanish","study","summer","swedish","synth-pop","tango","techno","trance","work-out","world-music"]
    # Dividi la stringa in "parole" basate su spazi o altri simboli, usando regex per trovare tutte le parole
    instruction, response = stringa.split('[/INST]')

    parole = re.findall(r'\b\w+\b', response)
    parole_trovate = []

    for parola in parole:

        # Pulisce ogni parola dai simboli, mantenendo solo caratteri alfabetici
        parola_pulita = re.sub(r'[^\w]', '', parola)
        # Controlla se una qualsiasi parola "pulita" è presente in genres
        for genere in genres:
            if re.search(r'\b' + re.escape(parola_pulita) + r'\b', genere, re.IGNORECASE):
                # Aggiungi alla lista delle parole trovate se corrisponde e non è già presente
                if genere not in parole_trovate:
                    parole_trovate.append(genere)
                break  # Interrompe il ciclo interno una volta trovata la corrispondenza
        if len(parole_trovate) == 3:
            break  # Interrompe il ciclo esterno una volta trovate 3 parole

    return parole_trovate

# print(parole_trovate)

## Flask

In [ ]:
# CODICE JAVASCRIPT PER FARE LA RICHIESTA AL MODELLO
# SOSTITUISCI CON L'URL DELLA SESSIONE COLAB
"""
let data = {
  text: "songs with a lot of energy from 70s 80s"
};

// Sostituisci 'URL_DEL_TUO_SERVER' con l'URL effettivo del server Flask
fetch('IL-TUO-URL/uppercase', {
  method: 'POST', // Metodo della richiesta
  headers: {
    'Content-Type': 'application/json', // Specifica che il corpo della richiesta è in formato JSON
  },
  body: JSON.stringify(data), // Trasforma l'oggetto `data` in una stringa JSON
})
.then(response => response.json()) // Converte la risposta in JSON
.then(data => {
  console.log('Success:', data); // Stampa i dati della risposta
})
.catch((error) => {
  console.error('Error:', error); // Stampa l'errore se qualcosa va storto
});

let data = {
    genres: ["pop", "rock", "indie"],
    access_token: "BQBpJxVg4Zwdj9e4hwCcybNQg3xxpkr4G8el9bGLZXrf_bVvn2Era8K4xQga8zfFZtL3m0qU5vAOA5OJTs2V-ms0IyvEWWlq1b3cCaLqnBbW5eb14Zg3_za08rUV5icbj9nHHGpSXdrdE44zUdheIT3DXwSpCvTk9Dn4IqmNu0A"
};

// Sostituisci 'URL_DEL_TUO_SERVER' con l'URL effettivo del server Flask
fetch('https://rgz5olderva-496ff2e9c6d22116-5000-colab.googleusercontent.com/createPlaylist', {
  method: 'POST', // Metodo della richiesta
  headers: {
    'Content-Type': 'application/json', // Specifica che il corpo della richiesta è in formato JSON
  },
  body: JSON.stringify(data), // Trasforma l'oggetto `data` in una stringa JSON
})
.then(response => response.json()) // Converte la risposta in JSON
.then(data => {
  console.log('Success:', data); // Stampa i dati della risposta
})
.catch((error) => {
  console.error('Error:', error); // Stampa l'errore se qualcosa va storto
});
"""

In [ ]:
from flask import Flask, send_from_directory, jsonify, request
import os
# from spotipy.oauth2 import SpotifyOAuth
import random

import requests
from requests.auth import HTTPBasicAuth

app = Flask(__name__, static_folder='/content/PlaylistCreator/frontend/build', static_url_path='')
#app = Flask(__name__)
# run_with_ngrok(app)

# Spotify API credentials
# SPOTIFY_CLIENT_ID = '08d52968d40a4cb999ddd47d784b5ac5'
# SPOTIFY_CLIENT_SECRET = '12683b0d2847456b975fb25c57cd1604'
# SPOTIFY_REDIRECT_URI = 'https://rgz5olderva-496ff2e9c6d22116-5000-colab.googleusercontent.com/'
# SCOPE = 'user-top-read playlist-modify-public user-read-private'

BASE_PROMPT = "[INST] genres: [acoustic,afrobeat,alt-rock,alternative,ambient,anime,black-metal,blues,bossanova,brazil,breakbeat,british,children,chill,classical,club,comedy,country,dance,dancehall,death-metal,deep-house,disco,disney,drum-and-bass,dub,dubstep,edm,electro,electronic,emo,folk,french,funk,garage,german,gospel,goth,groove,grunge,guitar,happy,hard-rock,hardcore,hardstyle,heavy-metal,hip-hop,holidays,house,idm,indian,indie,indie-pop,industrial,j-pop,jazz,k-pop,kids,latin,latino,metal,metal-misc,metalcore,minimal-techno,movies,new-age,opera,party,piano,pop,pop-film,post-dubstep,power-pop,progressive-house,psych-rock,punk,punk-rock,r-n-b,rainy-day,reggae,reggaeton,road-trip,rock,rock-n-roll,rockabilly,romance,sad,salsa,samba,show-tunes,singer-songwriter,sleep,songwriter,soul,soundtracks,spanish,study,summer,swedish,synth-pop,tango,techno,trance,work-out,world-music] As an assistant, identify the three most suitable genres from the list above for the following description: '"

@app.route('/genres', methods=['POST'])
def genres():
    data = request.json  # Assume che i dati inviati siano in formato JSON
    text = data.get('text', '')  # Ottiene la stringa dal campo 'text' del JSON ricevuto
    if text:
        model.eval()

        prompt = BASE_PROMPT + text + "' [/INST]"
        print("PLAYLIST DESCRIPTION: " + text)
        inputs = tokenizer(prompt, return_tensors="pt")

        outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=25)

        parole_trovate = extract_genres(tokenizer.batch_decode(outputs)[0])

        parole_trovate_str = ' '.join(parole_trovate)  # Converte la lista in una stringa
        print("GENERI IDENTIFICATI DAL MODELLO: ", parole_trovate)
        return jsonify({'text': parole_trovate_str})

    else:
        # Se non viene fornita alcuna stringa, invia un messaggio di errore
        return jsonify({'error': 'No text provided'}), 400

@app.route('/')
def serve():
    return send_from_directory(app.static_folder, 'index.html')

@app.route('/<path:path>')
def static_proxy(path):
    file_name = path.split('/')[-1]
    directory_name = os.path.join(app.static_folder, '/'.join(path.split('/')[:-1]))
    return send_from_directory(directory_name, file_name)


if __name__ == '__main__':
  app.run(host='0.0.0.0', port=8084)

## Use model

In [ ]:
# prompt = f""" [INST] genres: [acoustic,afrobeat,alt-rock,alternative,ambient,anime,black-metal,blues,bossanova,brazil,breakbeat,british,children,chill,classical,club,comedy,country,dance,dancehall,death-metal,deep-house,disco,disney,drum-and-bass,dub,dubstep,edm,electro,electronic,emo,folk,french,funk,garage,german,gospel,goth,groove,grunge,guitar,happy,hard-rock,hardcore,hardstyle,heavy-metal,hip-hop,holidays,house,idm,indian,indie,indie-pop,industrial,j-pop,jazz,k-pop,kids,latin,latino,metal,metal-misc,metalcore,minimal-techno,movies,new-age,opera,party,piano,pop,pop-film,post-dubstep,power-pop,progressive-house,psych-rock,punk,punk-rock,r-n-b,rainy-day,reggae,reggaeton,road-trip,rock,rock-n-roll,rockabilly,romance,sad,salsa,samba,show-tunes,singer-songwriter,sleep,songwriter,soul,soundtracks,spanish,study,summer,swedish,synth-pop,tango,techno,trance,work-out,world-music]
# As an assistant, identify the three most suitable genres from the list above for the following description:
# 'Nostalgic 80s hits that make you wish you were a teenager back then.' [/INST]
# """

# model.eval()
# inputs = tokenizer(prompt, return_tensors="pt")

# outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=25)
# print(tokenizer.batch_decode(outputs)[0])